In [1]:
import pandas as pd
import sys
sys.path.append('..')
import data_loader as dl
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def enumerate_y_peaks(row):
    enumeration = []
    Y_and_B = row["Matched Ion Series"].split(";")
    
    if len(Y_and_B) == 2: #if there are y and b peaks
        if "y" in Y_and_B[1]:
            peaks = Y_and_B[1]
        elif "y" in Y_and_B[0]:
            peaks = Y_and_B[0]
            
        peak_list = peaks.strip('][').split(",")

        for peak in peak_list:
            enumer_string = peak.split("+")[0]
            enumer = int(re.search(r'\d+', enumer_string).group(0))
            enumeration.append(enumer)
                
    else: #if there are either y or b peaks
        if "y" in Y_and_B[0]: #if it's a y peak
            peaks = Y_and_B[0]
            peak_list = peaks.strip('][').split(",")

            for peak in peak_list:
                enumer_string = peak.split("+")[0]
                enumer = int(re.search(r'\d+', enumer_string).group(0))
                enumeration.append(enumer)
        
        else: #if it's a b peak
            pass

        
    return enumeration  

In [3]:
def enumerate_b_peaks(row):
    enumeration = []
    Y_and_B = row["Matched Ion Series"].split(";")
    
    if len(Y_and_B) == 2: #if there are y and b peaks
        if "b" in Y_and_B[1]:
            peaks = Y_and_B[1]
        elif "b" in Y_and_B[0]:
            peaks = Y_and_B[0]
            
        peak_list = peaks.strip('][').split(",")

        for peak in peak_list:
            enumer_string = peak.split("+")[0]
            enumer = int(re.search(r'\d+', enumer_string).group(0))
            enumeration.append(enumer)
                
    else: #if there are either y or b peaks
        if "b" in Y_and_B[0]: #if it's a b peak
            peaks = Y_and_B[0]
            peak_list = peaks.strip('][').split(",")

            for peak in peak_list:
                enumer_string = peak.split("+")[0]
                enumer = int(re.search(r'\d+', enumer_string).group(0))
                enumeration.append(enumer)
        
        else: #if it's a y peak
            pass

        
    return enumeration    

In [4]:
def count_consecutive(row, col_name):
    peak_list = row[col_name]
    retlist = []
    count = 1
    # Avoid IndexError for  random_list[i+1]
    for i in range(len(peak_list) - 1):
        # Check if the next number is consecutive
        if peak_list[i] + 1 == peak_list[i+1]:
            count += 1
        else:
            # If it is not append the count and restart counting
            retlist.append(count)
            count = 1
    # Since we stopped the loop one early append the last count
    retlist.append(count)
    return retlist

In [5]:
def get_max_consec(row, col_name):
    return max(row[col_name])

In [6]:
def perc_ladder_peaks(row):
    peak_link_list = row['Ypeak_enum']
    consec_peak_list = row['consecutive_y_peaks']
    consec_peak_list = list(filter(lambda a: a != 1, consec_peak_list))
    if len(peak_link_list) == 0:
        perc = -1 #this is for when there no Y peaks
    else:
        perc = sum(consec_peak_list)/len(peak_link_list)
    return perc

In [25]:
def calc_consecutive_peaks(df):
    sc = df[['scan','Matched Ion Series', 'QValue']]
#     sc.assign(Bpeak_enum=enumerate_b_peaks, axis=1)
    sc["Bpeak_enum"] = sc.apply(enumerate_b_peaks, axis=1)
    sc['consecutive_b_peaks'] = sc.apply(lambda row: count_consecutive(row, 'Bpeak_enum'), axis=1)

    sc["Ypeak_enum"] = sc.apply(enumerate_y_peaks, axis=1)
    sc['consecutive_y_peaks'] = sc.apply(lambda row: count_consecutive(row, 'Ypeak_enum'), axis=1)

    sc['max_consecutive_b'] = sc.apply(lambda row: get_max_consec(row, "consecutive_b_peaks"), axis=1)
    sc['max_consecutive_y'] = sc.apply(lambda row: get_max_consec(row, "consecutive_y_peaks"), axis=1)

    sc['perc_in_ladder'] = sc.apply(perc_ladder_peaks, axis=1)
    sc = sc[['scan','max_consecutive_b','max_consecutive_y','perc_in_ladder', 'QValue']]

    

    return sc

In [14]:
sc2ng_rep1 = dl.clean_metamorph('2ng_rep1')
sc2ng_rep1

File Name   scan  Scan Retention Time  \
0      Ex_Auto_J3_30umTB_2ngQC_60m_1-calib  16668             52.52059   
1      Ex_Auto_J3_30umTB_2ngQC_60m_1-calib  27140             77.59262   
2      Ex_Auto_J3_30umTB_2ngQC_60m_1-calib  32708             91.05715   
3      Ex_Auto_J3_30umTB_2ngQC_60m_1-calib  33439             92.82384   
4      Ex_Auto_J3_30umTB_2ngQC_60m_1-calib  13306             44.50553   
...                                    ...    ...                  ...   
21696  Ex_Auto_J3_30umTB_2ngQC_60m_1-calib   9453             35.34856   
21697  Ex_Auto_J3_30umTB_2ngQC_60m_1-calib   9033             34.34908   
21698  Ex_Auto_J3_30umTB_2ngQC_60m_1-calib   7282             30.19998   
21699  Ex_Auto_J3_30umTB_2ngQC_60m_1-calib  15418             49.54085   
21700  Ex_Auto_J3_30umTB_2ngQC_60m_1-calib   6089             27.37473   

       Num Experimental Peaks  Total Ion Current  Precursor Scan Number  \
0                       101.0       3.352519e+05                  16649   
1                       155.0       1.258275e+06                  27124   
2                       133.0       3.061968e+05                  32695   
3                       200.0       8.794933e+05                  33418   
4                       165.0       4.429513e+05                  13293   
...                       ...                ...                    ...   
21696                   168.0       8.236731e+05                   9434   
21697                    84.0       2.344049e+06                   9016   
21698                   171.0       7.926368e+06                   7260   
21699                   200.0       1.643930e+06                  15406   
21700                   187.0       2.851842e+06                   6071   

       Precursor Charge  Precursor MZ  Precursor Mass   Score  ...  \
0                   2.0    1280.62814      2559.24172  28.510  ...   
1                   2.0    1108.03648      2214.05842  26.603  ...   
2                   3.0     832.05234      2493.13518  25.433  ...   
3                   3.0     856.05914      2565.15558  25.426  ...   
4                   3.0     729.02014      2184.03858  25.266  ...   
...                 ...           ...             ...     ...  ...   
21696               2.0     520.26380      1038.51304   5.010  ...   
21697               3.0     428.89074      1283.65040   5.009  ...   
21698               2.0     379.71063       757.40671   5.009  ...   
21699               2.0     615.27444      1228.53433   5.009  ...   
21700               3.0     406.85764      1217.55109   5.008  ...   

       Cumulative Target Cumulative Decoy    QValue Cumulative Target Notch  \
0                      6         0.000000  0.000000                       6   
1                     16         0.000000  0.000000                      16   
2                     36         0.000000  0.000000                      36   
3                     38         0.000000  0.000000                      38   
4                     51         0.000000  0.000000                      51   
...                  ...              ...       ...                     ...   
21696             129758     16879.154762  0.130082                  121682   
21697             129760     16881.154762  0.130095                  121683   
21698             129762     16883.154762  0.130105                    8070   
21699             129765     16883.154762  0.130105                  121686   
21700             129769     16884.654762  0.130108                  121688   

      Cumulative Decoy Notch  QValue Notch           PEP PEP_QValue  \
0                   0.000000      0.000000  2.443790e-05   0.000010   
1                   0.000000      0.000000  7.152557e-06   0.000003   
2                   0.000000      0.000000  1.764297e-05   0.000008   
3                   0.000000      0.000000  5.364418e-07   0.000000   
4                   0.000000      0.000000  6.210804e-05   0.000022   
...                      ..

In [30]:
all_files = ["2ng_rep1", "2ng_rep2", "2ng_rep3", "2ng_rep4", "2ng_rep5", "2ng_rep6", "0.2ng_rep1", "0.2ng_rep2", "0.2ng_rep3", "0.2ng_rep4", "0.2ng_rep5", "0.2ng_rep6"]

In [31]:
for file in all_files:
    df = dl.clean_metamorph(file)
    data = calc_consecutive_peaks(df)
    data.to_csv("peaks_files/" + file + "_peaks_data.csv")

<ipython-input-25-0653fa3f049d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc["Bpeak_enum"] = sc.apply(enumerate_b_peaks, axis=1)
<ipython-input-25-0653fa3f049d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc['consecutive_b_peaks'] = sc.apply(lambda row: count_consecutive(row, 'Bpeak_enum'), axis=1)
<ipython-input-25-0653fa3f049d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 